## Solution to Unfair Dice Mission 

For a full description of the problem, see [this page on check.io](http://www.checkio.org/mission/unfair-dice/)

To summarize, you are given a dice with a number of sides N (3 to 10) and an integer (1 to 18) on each side. The sum of all sides is S. Your task is to find a different die with sides that sum to S, that would win more than 50% of games played against the input dice. To determine a win, both die are rolled and if your die has a higher number, it wins. If the numbers are tied, both die are rolled again. If your number is lower, you lose. 

The challenge requires simply outputting one such die, if any exists.

I was interested in solving this using combinatorics to determine all posible valid die and then scoring each of these against the input and ranking them. My solution is below.

**For usage examples, scroll to the bottom of the notebook**


## Dependecies

division for Python 2.7 support.

In [1]:
from __future__ import division

## Various utility functions 

In [2]:
def partitionfunc(n,k,l=1):
    '''
    returns all lists of length k that sum to n with min entries of l
    
    n is the integer to partition, 
    k is the length of partitions, 
    l is the min partition element size
    '''
    if k < 1:
        raise StopIteration
    if k == 1:
        if n >= l:
            yield (n,)
        raise StopIteration
    for i in range(l,n+1):
        for result in partitionfunc(n-i,k-1,i):
            yield (i,)+result
            
def sort_lists_by(lists, key_list=0, desc=False):
    '''
    sorts multiples lists by the same key
    
    lists is a tuple of lists to sort
    key_list is the index of which list to sort by
    desc sets whether to ascend or descent sort
    '''
    return zip(*sorted(zip(*lists), reverse=desc,
                 key=lambda x: x[key_list]))

def numGreater(num,numList):
    '''counts the number of elements in numList which num is greater than'''
    return sum(1 for item in numList if num>item)

def sub(a,b):
    'subtract two lists of numbers elementwise'
    return [i - j for i, j in zip(a, b)]

def add(a,b):
    'add two lists of numbers elementwise'
    return [i + j for i, j in zip(a, b)] 


def winProbabilities(a,b):
    '''
    returns tuple (aProb,bProb)
    probability of dice a winning against dice b and vice-versa
    floating point numbers ranging 0.0 to 1.0
    
    a and b are input dice lists of the same length
    '''
    
    aProb = 0.0
    bProb = 0.0
    
    for aSide in a:
        aProb += 1.0/(len(a) ** 2) *  numGreater(aSide,b)
        
    for bSide in b:
        bProb += 1.0/(len(b) ** 2) *  numGreater(bSide,a)
    
    if aProb == bProb or (aProb == 0.0 and bProb == 0.0):
        return (0.50,0.50)
    else:
        return (aProb/(aProb+bProb),bProb/(aProb+bProb))

## The main solution function
This function **is not verbose**. It just spits out the answer.

In [3]:
def winning_die(die):
    '''
    This returns the best die to beat the input die based on the rules in the Unfair Dice challange
    The winning die is a list of integers. If no winner exists the output is []
    This function will not print any diagnostic information (see below verbose version)
    
    '''
    
    #Ensure die is sorted
    die = sorted(die)
    
    #Get some statistics about input die
    l = len(die)
    t = sum(die)
    biggest = max(die)
    smallest = min(die)
    
    #Check that sides are all integers
    dieInt = map(int,die)
    dieIntDiff = sub(die, dieInt) 
    zerosList = [0] * l
    
    # Basic checks for die validity
    if l>10 or l<3 or t>100 or biggest > 18 or smallest < 1 or dieIntDiff != zerosList:
        print("You input dice doest't meet the rules. Try another input.")
        return
    
    ###Begin game###
    
    # Obtain all possible valid die
    # Partitionfunc will return all lists of length l that sum to t with min entries of 1
    allDie = []
    for p in partitionfunc(t,l,1):
        allDie.append(list(p))
    
    # If not additional valid die exist, return []
    if(len(allDie)==1):
        return []
    
    # Remove original die from allDie list
    if die in allDie: allDie.remove(die)
    
    # Determine the win probability of all valid die against the input die
    # and sort the results
    probabilities = []
    for newDie in allDie:
        (winProb,loseProb) = winProbabilities(newDie,die)
        probabilities.append(winProb)
        
    allDieSorted, probabilitiesSorted = sort_lists_by((allDie,probabilities), key_list=1, desc=True)
    
    # Return the best result
    for d,p in zip(allDieSorted,probabilitiesSorted):
        if p>0.500001: #to eliminate rounding errors
            return d
        else:  
            #no solutions with better than 50% probability of winning
            return []
    

    

## The main solution function
This function **is verbose**. It provides various diagnostics and ways to investigate the solutions.

In [4]:
def winning_die_verbose(die,outputmax=10,printLosers=False):
    '''
    This version provides verbose output for examining results
    outputmax sets the number of winners to display
    printLosers enables/disables the output of the losing die
    the losing die are ALL printed
    '''
    
    #Ensure die is sorted
    die = sorted(die)
    
    #Get some statistics about input die
    l = len(die)
    t = sum(die)
    biggest = max(die)
    smallest = min(die)
    
    #Check that sides are all integers
    dieInt = map(int,die)
    dieIntDiff = sub(die, dieInt) 
    zerosList = [0] * l
    
    #Basic checks for die validity
    if l>10 or l<3 or t>100 or biggest > 18 or smallest < 1 or dieIntDiff != zerosList:
        print("You input dice doest't meet the rules. Try another input.")
        return
    
    
    ###Begin game###
    
    # Obtain all possible valid die
    # Partitionfunc will return all lists of length l that sum to t with min entries of 1
    allDie = []
    for p in partitionfunc(t,l,1):
        allDie.append(list(p))
    
    if(len(allDie)==1):
        print("There are no other die!")
        return []
    
    
    #Remove original die from allDie list
    if die in allDie: allDie.remove(die)
   
    
    #Determine the win probability of all valid die against the input die
    #Store win probabilities in list probabilities
    probabilities = []
    for newDie in allDie:
        (winProb,loseProb) = winProbabilities(newDie,die)
        probabilities.append(winProb)
    
    #Sort all the valid die by their win probabilities and store result in sorted lists
    allDieSorted, probabilitiesSorted = sort_lists_by((allDie,probabilities), key_list=1, desc=True)
    
    #Tally winners into count
    #Prepare a string to output the results
    count,countTies,countLosers = 0,0,0
    out,outLosers = "",""
    
    for d,p in zip(allDieSorted,probabilitiesSorted):
        if p>0.500001:
            if count < outputmax:
                out +=  str(d) + "\t wins %0.2f" % (100*p) + "% of games\n"
            count += 1
        else:
            theStr = "%0.2f" % (100-100*p)
            
            if printLosers:
                outLosers +=  str(d) 
            
            if theStr == "50.00":
                countTies +=1
                if printLosers:
                    outLosers += "\t ties the input\n"
            else:
                countLosers += 1
                if printLosers:
                    outLosers += "\t loses " + theStr + "% of games\n"

    #Print appropriate output based on number of winners
    validDie = len(allDieSorted)
    
    print(("{:10}{:8}{}{:7.2%}{}").format("Winners:",count,"  (",count/validDie,")"))
    print(("{:10}{:8}{}{:7.2%}{}").format("Ties:",countTies,"  (",countTies/validDie,")"))
    print(("{:10}{:8}{}{:7.2%}{}").format("Losers:",countLosers,"  (",countLosers/validDie,")"))
    print(("{:10}{:8}{}").format("Valid die:",validDie,"\n"))
    

    if count == 0:
        print("There are no better die!")
    elif count == 1:
        print("Here is the winner:\n" + out)
    else:
        print("Here are the top", min(count,outputmax), "winners:\n" + out)
    
    if printLosers and len(outLosers)>0:
            print("Here are the losing die:")
            print(outLosers)
    
    

## Example usage below

These are the test cases on the Unfair Dice description page

In [5]:
testCases = ([3, 3, 3, 3, 6, 6],[4, 4, 4, 4, 4, 4],[2, 2, 5, 5, 5, 5],[1, 1, 3],
            [1, 2, 3, 4, 5, 6],[2, 3, 4, 5, 6, 7],[1, 2, 3, 4, 5, 6])

for test in testCases:
    print("in :",test,"\nout:",winning_die(test),"\n")

in : [3, 3, 3, 3, 6, 6] 
out: [4, 4, 4, 4, 4, 4] 

in : [4, 4, 4, 4, 4, 4] 
out: [1, 4, 4, 5, 5, 5] 

in : [2, 2, 5, 5, 5, 5] 
out: [2, 2, 2, 6, 6, 6] 

in : [1, 1, 3] 
out: [1, 2, 2] 

in : [1, 2, 3, 4, 5, 6] 
out: [] 

in : [2, 3, 4, 5, 6, 7] 
out: [1, 1, 4, 7, 7, 7] 

in : [1, 2, 3, 4, 5, 6] 
out: [] 



**Let's examine the top 5 die for each case**

In [6]:
for test in testCases:
    print("-"*64+"\ninput:",test,"\n")
    winning_die_verbose(test,5)

----------------------------------------------------------------
input: [3, 3, 3, 3, 6, 6] 

Winners:        23  ( 11.62%)
Ties:           30  ( 15.15%)
Losers:        145  ( 73.23%)
Valid die:     198

Here are the top 5 winners:
[4, 4, 4, 4, 4, 4]	 wins 66.67% of games
[3, 4, 4, 4, 4, 5]	 wins 62.50% of games
[3, 3, 4, 4, 4, 6]	 wins 61.54% of games
[1, 4, 4, 4, 4, 7]	 wins 61.11% of games
[1, 4, 4, 4, 5, 6]	 wins 58.82% of games

----------------------------------------------------------------
input: [4, 4, 4, 4, 4, 4] 

Winners:        16  (  8.08%)
Ties:           46  ( 23.23%)
Losers:        136  ( 68.69%)
Valid die:     198

Here are the top 5 winners:
[1, 4, 4, 5, 5, 5]	 wins 75.00% of games
[1, 1, 5, 5, 5, 7]	 wins 66.67% of games
[1, 1, 5, 5, 6, 6]	 wins 66.67% of games
[1, 2, 5, 5, 5, 6]	 wins 66.67% of games
[1, 3, 5, 5, 5, 5]	 wins 66.67% of games

----------------------------------------------------------------
input: [2, 2, 5, 5, 5, 5] 

Winners:        18  (  9.09%)
Tie

**This die can be beat by any valid die (bad choice!)**

In [7]:
winning_die_verbose([1,1,1,10],20,True)

Winners:        17  (100.00%)
Ties:            0  (  0.00%)
Losers:          0  (  0.00%)
Valid die:      17

Here are the top 17 winners:
[2, 2, 2, 7]	 wins 75.00% of games
[2, 2, 3, 6]	 wins 75.00% of games
[2, 2, 4, 5]	 wins 75.00% of games
[2, 3, 3, 5]	 wins 75.00% of games
[2, 3, 4, 4]	 wins 75.00% of games
[3, 3, 3, 4]	 wins 75.00% of games
[1, 2, 2, 8]	 wins 69.23% of games
[1, 2, 3, 7]	 wins 69.23% of games
[1, 2, 4, 6]	 wins 69.23% of games
[1, 2, 5, 5]	 wins 69.23% of games
[1, 3, 3, 6]	 wins 69.23% of games
[1, 3, 4, 5]	 wins 69.23% of games
[1, 4, 4, 4]	 wins 69.23% of games
[1, 1, 2, 9]	 wins 60.00% of games
[1, 1, 3, 8]	 wins 60.00% of games
[1, 1, 4, 7]	 wins 60.00% of games
[1, 1, 5, 6]	 wins 60.00% of games



**A test case that can be beat by around 30% of valid die**

In [8]:
winning_die_verbose([2,17,17,17,17],20,False)

Winners:      2838  ( 29.75%)
Ties:          777  (  8.14%)
Losers:       5926  ( 62.11%)
Valid die:    9541

Here are the top 20 winners:
[3, 3, 18, 18, 28]	 wins 68.00% of games
[3, 3, 18, 19, 27]	 wins 68.00% of games
[3, 3, 18, 20, 26]	 wins 68.00% of games
[3, 3, 18, 21, 25]	 wins 68.00% of games
[3, 3, 18, 22, 24]	 wins 68.00% of games
[3, 3, 18, 23, 23]	 wins 68.00% of games
[3, 3, 19, 19, 26]	 wins 68.00% of games
[3, 3, 19, 20, 25]	 wins 68.00% of games
[3, 3, 19, 21, 24]	 wins 68.00% of games
[3, 3, 19, 22, 23]	 wins 68.00% of games
[3, 3, 20, 20, 24]	 wins 68.00% of games
[3, 3, 20, 21, 23]	 wins 68.00% of games
[3, 3, 20, 22, 22]	 wins 68.00% of games
[3, 3, 21, 21, 22]	 wins 68.00% of games
[3, 4, 18, 18, 27]	 wins 68.00% of games
[3, 4, 18, 19, 26]	 wins 68.00% of games
[3, 4, 18, 20, 25]	 wins 68.00% of games
[3, 4, 18, 21, 24]	 wins 68.00% of games
[3, 4, 18, 22, 23]	 wins 68.00% of games
[3, 4, 19, 19, 25]	 wins 68.00% of games



**Sequences beginning at 1 and increasing by 1 cannot be beat! **

In [9]:
winning_die_verbose([1,2,3,4,5,6,7,8,9],10,False)

Winners:         0  (  0.00%)
Ties:          909  ( 11.87%)
Losers:       6747  ( 88.13%)
Valid die:    7656

There are no better die!


**A test near the edge of the rule limits for the die**

In [10]:
winning_die_verbose([17,18,18,18,18],10,False)

Winners:      5935  ( 24.47%)
Ties:            0  (  0.00%)
Losers:      18324  ( 75.53%)
Valid die:   24259

Here are the top 10 winners:
[1, 19, 19, 19, 31]	 wins 80.00% of games
[1, 19, 19, 20, 30]	 wins 80.00% of games
[1, 19, 19, 21, 29]	 wins 80.00% of games
[1, 19, 19, 22, 28]	 wins 80.00% of games
[1, 19, 19, 23, 27]	 wins 80.00% of games
[1, 19, 19, 24, 26]	 wins 80.00% of games
[1, 19, 19, 25, 25]	 wins 80.00% of games
[1, 19, 20, 20, 29]	 wins 80.00% of games
[1, 19, 20, 21, 28]	 wins 80.00% of games
[1, 19, 20, 22, 27]	 wins 80.00% of games



**Enabling the output of losing die helps to examine patterns for winning strategy**

Patterns such as *small-small-high* seems to be pretty bad. However, you can sacrifice less than half the sides with small numbers in order to put high numbers on the other sides.

In [12]:
winning_die_verbose([1,2,3,5],10,True)

Winners:         3  ( 30.00%)
Ties:            2  ( 20.00%)
Losers:          5  ( 50.00%)
Valid die:      10

Here are the top 3 winners:
[2, 3, 3, 3]	 wins 58.33% of games
[1, 3, 3, 4]	 wins 53.85% of games
[2, 2, 3, 4]	 wins 53.85% of games

Here are the losing die:
[1, 2, 4, 4]	 ties the input
[2, 2, 2, 5]	 ties the input
[1, 1, 3, 6]	 loses 53.85% of games
[1, 1, 4, 5]	 loses 53.85% of games
[1, 2, 2, 6]	 loses 53.85% of games
[1, 1, 2, 7]	 loses 61.54% of games
[1, 1, 1, 8]	 loses 69.23% of games

